# 0. Imports and functions

In [ ]:
import pandas as pd
import pymssql
import yaml
from IPython.core.interactiveshell import InteractiveShell

from femsntl.datafiles import (
    CREDENTIALS_FILE,
    EXTERNAL_DIR,
    INTERMEDIATE_DIR,
    OUTPUT_DIR,
    PKL_FILE,
    PRIVATE_DATA_DIR,
    SAFETYPAD_DIR,
    SQL_DUMP_FILE,
)
from femsntl.themes import *
from femsntl.utils import clean_column_names, longform_crosstab

InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_columns", None)  # or 1000
pd.set_option("display.max_rows", None)  # or 1000
pd.set_option("display.max_colwidth", -1)

## Constants

In [ ]:
NTL_START_DATE = "2018-03-19"
NTL_END_DATE = "2019-03-01"

PULL_SQL_EVEN_IF_EXISTS = False
PULL_DBFILE_FROM_PKL = True
EXPORT_TO_SAFETYPAD = False
STORE_AS_PARQUET = False

# 1. Load data

Two versions:

- Data pulled from raw query similar to Nicole's: contains updated info we want like phone number and address; drawback is that it doesn't have the long ifelse chain that overrides the original disposition codes for certain IDs
- Data pulled from the Tableau data she sent in an email: contains the correct disposition codes


In [ ]:
# Load credentials
if not PULL_DBFILE_FROM_PKL:
    with open(CREDENTIALS_FILE, "rt") as cred_file:
        creds = yaml.load(cred_file)

    # Setup connection
    conn = pymssql.connect(**creds["cad_db"])

    # query text
    # CAD Query that loads our data
    ntl_summary_query = """
    SELECT
      ae.eid,
      ae.num_1,
      ae.sdts,
      ae.dgroup,
      ae.tycod,
      ae.typ_eng,
      ae.xdts,
      ae.ecbd_id,
      ae.status_code AS status_code, 
      ae.xcmt AS ae_xcmt, 
      ae.ssec,
      ae.ad_sec,
      ntl.cdts,
      ae.ds_ts,
      ce.edirpre,
      ce.estnum,
      ce.efeanme,
      ce.eapt, 
      ce.efeatyp,
      ce.edirsuf,
      ce.loc_com, 
      ce.ecompl,
      ntl.num_1 AS ntl_num_1,
      ntl.external_event_id,
      ntl.dispo,
      ntl.xcmt AS ntl_xcmt,
      ec.comm,
      cec.clname,
      cec.clrnum,
      cec.cstr_add
    FROM AGENCY_EVENT ae
    LEFT JOIN common_event_call cec ON ae.eid = cec.eid 
    LEFT JOIN common_event ce ON ae.eid = ce.eid 
    LEFT JOIN ntl_cache ntl ON ae.num_1 = ntl.num_1
    LEFT JOIN EVCOM ec ON
      ae.eid = ec.eid AND
      ec.COMM = 'NTL ** CANCEL REQUESTED BY ECBD. TRANSFER EVENT'
    WHERE
      LEFT(ae.sdts, 12) >= '201804190900' AND 
      (ntl.num_1 IS NOT NULL OR (ae.TYCOD LIKE '%NTL%' AND LEFT(ae.sdts, 8) < '20180531')) AND
      ISNULL(LEFT(ae.XCMT, 4), 'T') <> 'TEST' AND
      ISNULL(ae.XCMT, 'T') <> 'CBD TEST'
    """

    ntl_summary_raw = pd.read_sql_query(ntl_summary_query, conn)
    ntl_summary_raw.to_parquet(SQL_DUMP_FILE)

Below uses Nicole's query with two additions: 
- Adding in the left join to common_event_call which gives us names and phone numbers
- Adding in the left join to common_event which gives us more details on addresses and also the name of commonly-recognized locations (ecompl)

In future, could restrict date range so that it doesn't load more recent events

In [ ]:
# code to read in data from pkl or parquet backup rather than
# directly from DB
if SQL_DUMP_FILE.exists() and not PULL_SQL_EVEN_IF_EXISTS:
    ntl_summary_raw = pd.read_parquet(SQL_DUMP_FILE)
elif PULL_DBFILE_FROM_PKL:
    ntl_summary_raw = pd.read_pickle(PKL_FILE)


print(f"Number of rows in raw query {len(ntl_summary_raw)}")

# Pull in static file from Nicole
ntl_summary_clean = pd.read_csv(PRIVATE_DATA_DIR / "ntl_data_tableau.csv")
print(f"Number of rows in Nicole's tableau table is {len(ntl_summary_clean)}")

## difference reflects i think exclusions for testing/ineligibility

In [ ]:
## Fix column names for cleaned data
ntl_summary_clean.columns = clean_column_names(ntl_summary_clean.columns)

In [ ]:
## merge on the basis of agency_event with the data from the raw query
## this preserves the info from the raw database but includes the tableau-coded
## overriden event codes
ntl_analytic = ntl_summary_raw.merge(
    ntl_summary_clean[
        ["agency_event", "reported_event_status", "event_status", "ntl_id"]
    ],
    left_on="num_1",
    right_on="agency_event",
    how="left",
)

# 2. Initial data cleaning and merging in Safety PAD

- Subset observations to dates of study
- Create informative names for disposition codes

In [ ]:
## pull date and make date-time object
ntl_analytic["date"] = pd.to_datetime(
    ntl_analytic.sdts.str.extract("(?P<date_str>201[8|9]\d{10})").date_str,
    format="%Y%m%d%H%M%S",
)

In [ ]:
## subset to calls within the evaluation period
ntl_summary_eval = ntl_analytic[
    (ntl_analytic.date >= NTL_START_DATE) & (ntl_analytic.date < NTL_END_DATE)
].copy()

print(f"Rows in raw data: {len(ntl_analytic)}")
print(f"Rows in NTL range: {len(ntl_summary_eval)}")

In [ ]:
## create variable just truncated to the month level
ntl_summary_eval["month_year"] = ntl_summary_eval.date.dt.to_period("M")
ntl_summary_eval["month_year_day"] = ntl_summary_eval.date.dt.to_period("D")

## 2.1 Descriptive one: number of observations in different study groups over time


In [ ]:
## count of disposition code by month
dispo_bymonth = pd.crosstab(ntl_summary_eval.event_status, ntl_summary_eval.month_year)
all_event_codes = ntl_summary_eval.event_status.unique().tolist()

ntl_handled_codes = [
    code for code in all_event_codes if "NTL Handled" in code or "NTL - Other" in code
]
field_requested_codes = [
    code for code in all_event_codes if "Field Requested NTL" in code
]
transfer_codes = [code for code in all_event_codes if "Transfer from NTL" in code]
control_codes = ["Study Reject"]
out_of_band_codes = ["Request outside the hours of operation"]
other_treatment_codes = [
    "Incompatible Code from AMR",
    "RESP EXPIRED",
    "SERVICE NOT AVAILABLE",
]  # new tx codes as indicated by nicole
treatment_codes = (
    ntl_handled_codes + field_requested_codes + transfer_codes + other_treatment_codes
)

print(
    "We are assuming that the treatment group event codes are:\n  * "
    + "\n  * ".join(treatment_codes)
)

other_codes = [
    code
    for code in all_event_codes
    if code not in (control_codes + treatment_codes + out_of_band_codes)
]

In [ ]:
## summarize broad categories
ntl_summary_eval["dispo_broad"] = "Other"
for label, group in [
    ("NTL treatment", treatment_codes),
    ("NTL control", control_codes),
    ("Outside NTL hours", out_of_band_codes),
]:
    ntl_summary_eval.loc[
        ntl_summary_eval.event_status.isin(group), "dispo_broad"
    ] = label

In [ ]:
## summarize braod categories by month
dispobroad_bymonth = pd.crosstab(
    ntl_summary_eval.dispo_broad, ntl_summary_eval.month_year
)
dispobroad_bymonth_long = longform_crosstab(dispobroad_bymonth, grouping_var="dispo")
dispo_txcont = dispobroad_bymonth_long.loc[
    dispobroad_bymonth_long.dispo.isin(["NTL control", "NTL treatment"])
].copy()
overall_trends = (
    ggplot(dispo_txcont, aes(x="date", y="value", color="factor(dispo)"))
    + geom_point()
    + geom_line()
    + standard_background
    + theme(
        axis_text_x=element_text(angle=90),
        legend_position=(0.5, 0.7),
        legend_background=element_blank(),
    )
    + labs(color="Broad Group")
    + ylab("Count of calls by month")
    + xlab("Month and year")
    + scale_color_manual(values=(CONTROL_COLOR, TREATMENT_COLOR))
)

ggsave(
    plot=overall_trends,
    filename=OUTPUT_DIR / "overall_trends_txcont.png",
    dpi=300,
    verbose=False,
)
overall_trends

## 2.2 Get IDS of NTL participants to search for them via safetypad interface and export their data

In [ ]:
study_groups = ["NTL control", "NTL treatment"]
ntl_participants = ntl_summary_eval[
    ntl_summary_eval.dispo_broad.isin(study_groups)
].copy()
n_total = len(ntl_participants.num_1.unique())
n_tx = len(
    ntl_participants.num_1[ntl_participants.dispo_broad == "NTL treatment"].unique()
)
n_control = len(
    ntl_participants.num_1[ntl_participants.dispo_broad == "NTL control"].unique()
)


print("Total NTL participants (unique calls; not unique people):", n_total)
print("Total treatment:", n_tx)
print("Total control:", n_control)
print(f"Percent is treatment: {n_tx / n_total * 100:.1f}%")

In [ ]:
## newly added- get ids of those referred to ntl (but ended up outside of caller hours;
## useful for repeat calls analysis because we count as a repeat call if they:
##   1) have an ntl code, but
##   2) call outside the study window
ntl_nonparticipants = ntl_summary_eval[
    ~ntl_summary_eval.num_1.isin(ntl_participants.num_1)
]
print(
    "Number of incidents NTL eligible but not randomized:",
    len(ntl_nonparticipants.num_1.unique()),
)
print("e.g. because outside study hours")

### 2.2.1 Get IDs in batches of 1000 to export to safety pad

#### 2.2.1.1: ids for treatment and control group

In [ ]:
if EXPORT_TO_SAFETYPAD == True:

    unique_ids = ntl_participants.num_1.unique()

    print("Unique NTL ids:", len(unique_ids))
    print("Total calls in treatment and control:", n_total)

    ## test with first 1000
    ## site: https://dcfems.safetypad.com/index.php
    ## formatting: no quotes, separated by comma
    store_id_batches = [
        ", ".join(unique_ids[left : left + 1000])
        for left in range(0, len(unique_ids), 1000)
    ]

    unique_ids_nonparticipants = ntl_nonparticipants.num_1.unique()

    print("Unique NTL ids in nonparticipant calls:", len(unique_ids_nonparticipants))
    print("Total nonparticipant calls:", len(ntl_nonparticipants))

    nonparticipant_store_id_batches = [
        ", ".join(unique_ids_nonparticipants[left : left + 1000])
        for left in range(0, len(unique_ids_nonparticipants), 1000)
    ]

In [ ]:
## previous step of searching ids in safety padresults in 6 files with following naming convention
## safetypad_idsearch_batch*.csv (after xls conversion bc struggled with corrupted file format)

## concatenate the CSVs
safetypad_files = list(SAFETYPAD_DIR.glob("safetypad_idsearch_batch*.csv"))

all_safetypad_df = pd.concat([pd.read_csv(filename) for filename in safetypad_files])
all_safetypad_df.columns = clean_column_names(all_safetypad_df.columns)

### 2.2.2 For ones who are clinical referral, self care, or others, write list of ids for AMR to search for 

Note: just included the ones in tx or control group; not other ids

In [ ]:
## create indicator in main data
ntl_participants["is_in_safetypad"] = ntl_participants.num_1.isin(
    all_safetypad_df.incident_number
)

## summary
amr_statuses = ["NTL Handled - RSC", "NTL Handled - Clinical Referral", "NTL - Other"]
print(
    "Number study participants in safety pad:",
    ntl_participants[ntl_participants.is_in_safetypad].num_1.nunique(),
)
print(
    "Number study participants *not* in safety pad safety pad:",
    ntl_participants[~ntl_participants.is_in_safetypad].num_1.nunique(),
)

## disposition codes of those not in safety pad
ntl_participants_notinsafetypad = ntl_participants[~ntl_participants.is_in_safetypad]

## get ids of ones who are RSC, clin. referral, or other
ntl_participants_notinsafetypad_checkwithAMR = ntl_participants_notinsafetypad[
    ntl_participants_notinsafetypad.event_status.isin(amr_statuses)
]

## about 1000 participants, write their name and dates to csv file
ntl_participants_forAMR = ntl_participants_notinsafetypad_checkwithAMR[
    ["num_1", "date", "event_status"]
]

print(
    f"Asking AMR to look for {ntl_participants_forAMR.num_1.nunique()}"
    " non-safety PAD ids who are in self-care, clin referral, or other"
)

## AMR participants
ntl_participants_forAMR.to_csv(
    EXTERNAL_DIR / "ntl_participants_forAMR.csv", index=False
)

### 2.2.3: for ones who are other statuses and not cancelled, write ids for Nicole or FEMs to search for

Note: included the "other" category in both IDs sent to AMR and ids to send to usc

In [ ]:
event_statuses_toask_OUC = [
    col
    for col in ntl_participants_notinsafetypad.event_status.unique()
    if col
    not in [
        "NTL Handled - RSC",
        "NTL Handled - Clinical Referral",
        "NTL - Other",
        "NTL Handled - Canceled",
    ]
]

ntl_participants_notinsafetypad_checkwithOUC = ntl_participants_notinsafetypad[
    ntl_participants_notinsafetypad.event_status.isin(event_statuses_toask_OUC)
]

ntl_participants_forOUC = ntl_participants_notinsafetypad_checkwithOUC[
    ["num_1", "date", "event_status"]
]

## OUC participants
ntl_participants_forOUC.to_csv(
    EXTERNAL_DIR / "ntl_participants_forOUC.csv", index=False
)

### 2.2.4: do a left join between 1) CAD and 2) safety PAD for the purposes of repeat calls analysis

This means we'll retain all participants, but 1300 or so will be missing safetyPAD info

In [ ]:
## also read in safetypad data on non participants
## since that could count as a repeat call for a participant
safetypad_nonparticipants = pd.read_csv(
    SAFETYPAD_DIR / "safetypad_idsearch_nonparticipants.csv"
)
safetypad_nonparticipants.columns = clean_column_names(
    safetypad_nonparticipants.columns
)

all_safetypad_df_withnon = pd.concat([all_safetypad_df, safetypad_nonparticipants])

In [ ]:
## left join safety pad to all ntl (so includes participants (tx and control) and those who called but werent in tx or control)
ntl_withsafetypad = ntl_summary_eval.merge(
    all_safetypad_df_withnon, left_on="num_1", right_on="incident_number", how="left"
)

print("Number of incidents left join:", len(ntl_participants))
print(
    "Number of incidents after left join (larger since multiple rows for each status):",
    len(ntl_withsafetypad),
)

### Clean types before dumping

In [ ]:
# Convert timestamps before dumping
for col in ["sdts", "XDTS", "cdts", "ds_ts"]:
    ntl_withsafetypad[col] = pd.to_datetime(
        ntl_withsafetypad[col].map(lambda x: x[:-2], na_action="ignore"),
        format="%Y%m%d%H%M%S",
    ).dt.tz_localize("US/Eastern", ambiguous=False)

In [ ]:
for col in [
    "en_route",
    "at_scene",
    "at_patient",
    "depart_scene",
    "at_destination",
    "in_service",
]:
    ntl_withsafetypad[col] = pd.to_datetime(
        ntl_withsafetypad[col], format="%m/%d/%Y %H:%M"
    ).dt.tz_localize("US/Eastern", ambiguous=False)

In [ ]:
# Parquet can't handle periods
for col in ["month_year", "month_year_day"]:
    ntl_withsafetypad[col] = ntl_withsafetypad[col].astype(str)

In [ ]:
# Fixup zip codes
ntl_withsafetypad["zip_code"] = ntl_withsafetypad.zip_code.map(
    lambda x: x if isinstance(x, int) else int(x[:5]), na_action="ignore"
)

In [ ]:
## write to parquet or pkl
if STORE_AS_PARQUET:
    ntl_withsafetypad.to_parquet(INTERMEDIATE_DIR / "ntl_withsafetypad.parquet")
else:
    ntl_withsafetypad.to_pickle(INTERMEDIATE_DIR / "ntl_withsafetypad.pkl")